# HydroHomies Plots
In this notebook, the plots, figures and also some explanations or details about each of them are being presented.  

To clarify plots, please follow this order:
- Title for each plot is mandatory
- Analysis must be written 
- legends are manedatory

### Importing the needed modules

In [ ]:
import yaml
import pandas as pd
import numpy as np
from scipy.stats import sem

from bokeh.plotting import figure, show
from bokeh.io import show, output_notebook
from bokeh.transform import dodge, factor_cmap
from bokeh.models import ColumnDataSource, FactorRange, Whisker, Range1d
from bokeh.transform import factor_cmap

from bokeh.layouts import gridplot
output_notebook()

import panel as pn
pn.extension()

import plotting_functions as pf

import hvplot.pandas

### Loading all data


In [ ]:
with open('config.yaml') as stream:
    config = yaml.safe_load(stream)

### Cleaning (Digit Span Raw Data)

In [ ]:
def clean_digit_span(raw_df):
    '''
    Function to clear the raw digit span results obtained from the online test. 
    It keeps the rows and columns that are informative, and makes some basic 
    operations to obtain informative variables.
    
    Arguments:
    raw_df: Dataframe obtained after reading the digitspan excel file with the 
    results.

    Returns:
    A clean and processed dataframe.
    
    Author:
    Karina Diaz 
    '''
    # Select the sequence length data from the raw data and create a dataframe
    seq_length_df = raw_df[raw_df[1].astype(str).str.match(r'\d+')]

    # Get the value of the longest sequence remembered. Is the column with index 2
    longest = seq_length_df[2]
    longest = longest.tolist()

    # Get the number of errors made. Is the column with index 3
    error_number = seq_length_df[3]
    error_number = error_number.tolist()

    # Select the rows with the click stimulus data
    click_stim_df = raw_df[raw_df[1]=='clickedStim']
    click_stim_df.size

    # Calculate the number of clicks made by the participant.
    clicks_observed = click_stim_df.count(axis=1) - 3 
    # 3 is subtracted because there are 3 non-informative values in the data 
    # that should not be taken into account.
    clicks_observed = clicks_observed.tolist()

    # Calculate the number of clicks that the participant should have made
    clicks_expected =  pd.to_numeric(longest) + 1
    clicks_expected = clicks_expected.tolist()

    # Create a new dataframe with all the values calculated above
    clean_data = pd.DataFrame(data ={'seq length':longest,
                        'errors': error_number,
                        'clicks expected': clicks_expected,
                        'clicks observed':clicks_observed})

    # Return the new dataframe
    return clean_data

### Data Integration For Each Test

A function is created to get all the data in dataframes. The data of each test (so of all participant) is merged/concatenated to get a big dataframe, which can easily used later for plotting

In [ ]:
def create_merged_df(config_dict):
    """
    Reads all of the files and creates a dataframe from it.
    It then concatenates each dataframe according to the test, 
    to obtain one big dataframe per test (and one health data)

    Args:
        config_dict (dict): dictionary with all of the filepaths

    Returns:
        dict: dictionary with all the merged dataframes, 
              where the key is the file name and the value
              the merge dataframe
              
    Author(s):
        Job Maathuis
        Hooman Bardo
    """
    data_dict = {}

    # read the files 
    for test, file in config_dict.items():
        df_dict = pd.read_excel(file, sheet_name=None, header=None)

        for session, df in df_dict.items():

            # extracting the participant name and type name
            participant = test.split('_')[-1]
            test_name = test.split('_')[0]

            # extracting repeat number and making its column except for personal health data
            try:
                type, repeat = session.split('_')
                df.insert(0, 'repeat', repeat)

            # healht data has a different structure, solving by taking session number
            except ValueError:
                type = session

            # Running function to clean digit span data
            if test_name == 'digit':
                df = clean_digit_span(df.iloc[3:])
                df.insert(0, 'repeat', repeat)
            
            # verbal fluency test contains header
            elif test_name =='verbal':
                df = df.iloc[1:]

            # inserting the type and participant columns
            df.insert(0, 'type', type)
            df.insert(0, 'participant', participant)
 
            # concatenating data frames of each test
            if test_name not in data_dict:
                data_dict[test_name] = df
            else:
                data_dict[test_name] = pd.concat([data_dict[test_name], df])
    
    return data_dict

data_dict = create_merged_df(config)

---

### Personal health data plots

In [ ]:
# creating personal dataframe
def create_personal_dataframe():
    p_df = data_dict["personal"].copy()
    p_df.drop(0, inplace=True)
    p_df.rename(columns={
        0: "session",
        1: "time",
        2: "heartrate",
        3: "calories",
        4:"temperature",
        5:"body weight",
        6: "muscle%",
        7: "fat%",
    }, inplace=True)
    p_df = p_df[[
        "participant",
        "type",
        "session",
        "time",
        "heartrate",
        "calories",
        "temperature",
        "body weight",
        "muscle%",
        "fat%"
    ]]
    
    # fill missing and not correct values with the correct one.
    p_df["heartrate"] = pd.to_numeric(p_df["heartrate"],errors='coerce')
    p_df["heartrate"] = p_df["heartrate"].fillna(85)
    p_df['session'] = p_df['session'].fillna(2)
    p_df["calories"] = p_df["calories"].fillna(1118)
    p_df["temperature"] = p_df["temperature"].fillna(36.4)
    
    p_df = p_df.astype({'heartrate': 'float', 'calories': 'float', 'temperature': 'float',
                       'body weight': 'float', 'fat%': 'float', 'muscle%': 'float'})

    return p_df

personal_df = create_personal_dataframe()

In [ ]:
# variables for plots to make them uniform
participants = ['red', 'orange', 'green', 'blue', 'pink']
colors      = ['salmon', 'skyblue']
line_color  = 'black'
plot_width  = 600
plot_height = 400

In [ ]:
def show_personal_plot(participant='pink', target = 'calories'):
    '''
    This function plots line_plots based on each participants and their health data
    
    Parameters: participant: 'green','pink', 'orange','blue','red'
                target: "heartrate", "calories", "temperature"
    
    Return: line plot 
    Authors: Roya
             Mahdiye
    '''
    target_units = {'heartrate': 'bpm', 'calories': 'kcals', 'temperature': '°C'}

    personal_df = create_personal_dataframe()
    personal_df = personal_df[personal_df["participant"] == participant]

    p = figure(x_range = [personal_df['time'].min(),personal_df['time'].max()*1.02], 
               y_range = [personal_df[target].min()*.5,personal_df[target].max()*1.3],
               width=600, height=400, title=f'linechart of {target} data of participant {participant}', 
               x_axis_label="time (minutes)", y_axis_label=f'{target} ({target_units[target]})')

    x = personal_df["time"].unique().tolist()
    y1 = personal_df[(personal_df["type"] == "dehydration") & (personal_df["session"] == 1)][target].tolist()
    y2 = personal_df[(personal_df["type"] == "dehydration") & (personal_df["session"] == 2)][target].tolist()
    y3 = personal_df[(personal_df["type"] == "control") & (personal_df["session"] == 1)][target].tolist()
    y4 = personal_df[(personal_df["type"] == "control") & (personal_df["session"] == 2)][target].tolist()
    
    # fitbit calories burned is an additive value of the whole day
    # setting the calories values to start at 0 
    if target == 'calories':
        y1 = np.array(y1) - np.min(y1)
        y2 = np.array(y2) - np.min(y2)
        y3 = np.array(y3) - np.min(y3)
        y4 = np.array(y4) - np.min(y4)
        # resetting the yrange
        p.y_range = Range1d(0, max([*y1, *y2, *y3, *y4])*1.3)
        
    # add multiple renderers
    p.line(x, y1, legend_label="dehydration1", color="blue", line_width=2)
    p.line(x, y2, legend_label="dehydration2", color="red", line_width=2)
    p.line(x, y3, legend_label="control1", color="green", line_width=2)
    p.line(x, y4, legend_label="control2", color="orange", line_width=2)
    p.legend.location = 'top_left'
    return p

# participants =['green','pink', 'orange','blue','red']
targets = ["heartrate", "calories", "temperature"]
personal_plot = pn.interact(show_personal_plot, participant=participants, target = targets)
personal_plot

In [ ]:
def total_bar_personal(target):
    '''
    This function plots a bar_plot based on the target.
    
    parameter: target: one of personal data like:
    "heartrate", "calories", "temperature", "body weight", "fat%", "muscle%"
    
    return: bar_plot
    Author: Mahdiye
    '''
    
    personal_df = create_personal_dataframe()
    df = personal_df

    #create a list of different session types
    types = list(df['type'].unique())
    
    dff = df.groupby(['participant','type']).mean().reset_index()
    
    # create a list of participants
    participants = list(dff['participant'].unique())

    #create two list of reaction time regarding session types
    control_mean = list(dff[dff['type'] =='control'][target])
    dehydration_mean = list(dff[dff['type'] =='dehydration'][target])

    #create a dictionary of 3 keys and values and then convert into a dataframe
    data = {'participants' : participants,
            'control'   : control_mean,
            'dehydration'   : dehydration_mean,
            }
    data = pd.DataFrame(data)

    palette =  ["skyblue", "salmon"] #colors

    # create a list like:
    # [ ("blue", "control"), ("Ablue", "dehydration"), ("red", "control"), ("red", "dehydration"), ... ]
    x = [ (participant, test) for participant in participants for test in types ]
    counts = sum(zip(data['control'], data['dehydration']), ()) # like an hstack

    source = ColumnDataSource(data=dict(x=x, counts=counts))
    # plot
    p = figure(x_range=FactorRange(*x), y_range=[0, data['dehydration'].max()*1.3], width=600, height=400, 
                title='Average of '+ target,
                y_axis_label="Reaction time(milliseconds)",
                x_axis_label="participant, session type")

    p.vbar(x='x', top='counts', width=1, source=source, line_color="black",
           fill_color=factor_cmap('x', palette=palette, factors=types, start=1, end=2))

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None

    return p

In [ ]:
def calculate_personal_standard_error(target):
    '''
    This function calculates upper and lower of whisker for error limits.
    
    parameter: target: one of personal data like:
    "heartrate", "calories", "temperature", "body weight", "fat%", "muscle%"
    
    return: a dataframe contains mean of target, upper and lower columns
    
    Author: Roya (The main author)
            Mahdiye (I had to change some parts to be run for my codes) 
    '''
    personal_df = create_personal_dataframe()
    df = personal_df
    df_mean = df.groupby(by=["participant", "type"]).agg(mean=(target, "mean"))
    df_se = df.groupby(by=["participant", "type"]).agg(se=(target, "sem"))
    upper = df_mean["mean"] + 1.96 * df_se["se"]
    lower = df_mean["mean"] - 1.96 * df_se["se"]
    data = pd.concat([upper.rename("upper"), lower.rename("lower")], axis=1)
    return data

def plot_standard_error(plot, data):
    '''
    This function plots Whiskers and add it to main bar plot
    
    Parameters: plot= the given bar plot
    
    Returns: plot with whiskers
    
    Autor: Roya
    '''
    x = list(data.index.values)
    data_map = {
        'x': x,
        'upper': data["upper"].tolist(),
        'lower': data["lower"].tolist()

        }
    source = ColumnDataSource(data=data_map)

    w = Whisker(source=source, base="x", upper="upper", lower="lower", 
                line_color='black', level="overlay")
    w.upper_head.line_color = 'black'
    w.lower_head.line_color = 'black'
    w.upper_head.size = w.lower_head.size = 20
    plot.add_layout(w)
    return plot

def personal_plot_error_bar(target):
    '''
    This function plots bar_plots with errors on them.
    
    parameter: target: one of personal data like:
    "heartrate", "calories", "temperature", "body weight", "fat%", "muscle%"
    
    return: the main bar plot with errors, related to the target
    
    Author: Roya (The main author)
            Mahdiye (I had to change some parts to be run for my codes) 
    '''
    data_se = calculate_personal_standard_error(target)
    p = total_bar_personal(target)
    p = plot_standard_error(p, data_se)
    return p
targets = ["heartrate", "calories", "temperature", "body weight", "fat%", "muscle%"]
personal_error_plot = pn.interact(personal_plot_error_bar, target = targets)
personal_error_plot

### Dehydration percentage

In [ ]:
# Obtain personal data
df_health = data_dict['personal']
# Theres one missing value in the green session. Fill it with the correct value
df_health[0] = df_health[0].fillna(2)

def dehydration_percentage(df_health):
    '''
    Function to obtain the % of dehydration per participant per session.
    
    Arguments:
    df_health: Dataframe with all the participants personal data. 

    Returns:
    A dataframe with the body weights and % of dehydration achived by every
    participant in every session.
    
    Author:
    Karina Diaz
    '''
    # Keep only the body weight data and drop NaN
    body_weight = df_health[['participant','type', 0, 5]].dropna()
    # drop the rows with strings on them (the ones with index ==0 )
    body_weight = body_weight[body_weight.index!=0]
    body_weight.rename(columns={0:'session', 5:'body weight'}, inplace=True)
    body_weight = body_weight.astype({'body weight':'float'})

    # calculate percentage of dehydration
    body_weight['dehydration %'] = (body_weight.groupby(
                                    ['participant','type','session']
                                    )['body weight'].pct_change()) * 100
    return body_weight

In [ ]:
# Runing the function from above # Mahdiye
dehydration_percentage = round(dehydration_percentage(df_health).dropna(),2)
dehydration_percentage.drop(dehydration_percentage.columns[3], axis=1, inplace=True)
dehydration_percentage = dehydration_percentage[dehydration_percentage['type']=='dehydration']
dehydration_percentage = dehydration_percentage.T
dehydration_percentage.columns = dehydration_percentage.iloc[0] # consider the first row as header
dehydration_percentage = dehydration_percentage[1:]
dehydration_percentage

---

## Flanker Test Analysis

#### Data Wrangling

In [ ]:
# creating Flanker dataframe
def create_flanker_dataframe():
    """
    cleaning data in order to be prepared for analysis step

    Returns:
        flanker_df(Dataframe) : Data of flanker test which is ready for proceesing.
              
    Author(s):
        Roya Gharehbeiklou
    """
    flanker_df = data_dict["flanker"].copy()
    #accroding to the concept of test, name of columns are changed 
    flanker_df.rename(columns={0: "pattern", 1: "expression", 2: "correctness", 3: "response-time"}, inplace=True)
    #unused information for this test which was created in data integration step.
    flanker_df.drop(columns=[4, 5, 6, 7, 8], inplace=True)
    #Replacing contents of correctness column
    flanker_df["correctness"] = flanker_df["correctness"].replace(1, "correct")
    flanker_df["correctness"] = flanker_df["correctness"].replace(2, "incorrect")
    flanker_df["correctness"] = flanker_df["correctness"].replace(3, "not-answer")
    flanker_df["correctness"] = flanker_df["correctness"].fillna("not-answer")
    #Because of NaN values that may happend in raw data
    flanker_df['response-time'].fillna((flanker_df['response-time'].mean()), inplace=True)
    flanker_df['expression'].fillna(0, inplace=True)
    return flanker_df

flanker_df = create_flanker_dataframe()
# summery of data for one user with correct answer type
flanker_df[(flanker_df['correctness'] == 'correct') & (flanker_df['participant'] == 'red')]

#### Data Analysis

In [ ]:
def flanker_calculate_counts(flanker_df, answer_type="correct"):
    """
    Counts the number of answeres for each participants per session
     
    Arguments:
        flanker_df(Dataframe): Dataframe with all the participants flanker test data. 
        answer_type(object): 'correct', 'incorrect','not-answer'
        
    Returns:
        data(Dataframe) : Average number of answers for each participant in
                          dehydaration and control.
              
    Author(s):
        Roya Gharehbeiklou
    """
    flanker_df = flanker_df[flanker_df["correctness"] == answer_type]
    flanker_df = flanker_df.groupby(["participant", "type", "repeat"])["correctness"].count().reset_index()
    data = flanker_df.groupby(by=["participant", "type"])["correctness"].mean()

    return data

def calculate_percentage(flanker_df, answer_type="correct"):
    """
    The number of answeres(percentagewise) for each participants per session
     
    Arguments:
        flanker_df(Dataframe): Dataframe with all the participants flanker test data. 
        answer_type(object): 'correct', 'incorrect','not-answer'
        
    Returns:
        data(Dataframe) : Average number of answers for each participant in
                          dehydaration and control.
              
    Author(s):
        Roya Gharehbeiklou
    """
    # calculate all number of questions per session
    df_all = flanker_df.groupby(["participant", "type", "repeat"]).agg(count=("correctness", "count"))
    # calculate the number of answer type chosen
    flanker_df = flanker_df[flanker_df["correctness"] == answer_type]
    df_correct = flanker_df.groupby(["participant", "type", "repeat"]).agg(count=("correctness", "count"))

    # Average calculation
    flanker_df = round(df_correct["count"] * 100 / df_all["count"], 2).rename("percentage").reset_index().fillna(0)
    data = flanker_df.groupby(by=["participant", "type"])["percentage"].mean()
    return data

def flanker_calculate_responce_time(flanker_df):
    """
    Counts the number of answeres for each participants per session
     
    Arguments:
        flanker_df(Dataframe): Dataframe with all the participants flanker test data. 
        
    Returns:
        data(Dataframe) : Average number of answers for each participant in
                          dehydaration and control.
              
    Author(s):
        Roya Gharehbeiklou
    """
    data = flanker_df.groupby(by=["participant", "type"])["response-time"].mean()

    return data
def calculate_flanker_standard_error(flanker_df, answer_type, column_name="correctness"):
    """
    This function calculate SE for each participants per session
     
    Arguments:
        flanker_df(Dataframe): Dataframe with all the participants flanker test data. 
        answer_type(str): 'correct', 'incorrect','not-answer'
        column_name(str): which can be any column but we use it for "response-time" and "correctness"
    Returns:
        data(Dataframe) : upper an lower of Error for each participants per session
              
    Author(s):
        Roya Gharehbeiklou
    """
    if column_name == "correctness":
        flanker_df = flanker_df[flanker_df["correctness"] == answer_type]
        flanker_df = flanker_df.groupby(["participant", "type", "repeat"])["correctness"].count().reset_index()
        df_mean = flanker_df.groupby(by=["participant", "type"]).agg(mean=("correctness", "mean"))
        df_se = flanker_df.groupby(by=["participant", "type"]).agg(se=("correctness", "sem"))

    elif column_name == "response-time":
        df_mean = flanker_df.groupby(by=["participant", "type"]).agg(mean=("response-time", "mean"))
        df_se = flanker_df.groupby(by=["participant", "type"]).agg(se=("response-time", "sem"))
        
    upper = df_mean["mean"] + 1.96 * df_se["se"]
    lower = df_mean["mean"] - 1.96 * df_se["se"]
    data = pd.concat([upper.rename("upper"), lower.rename("lower")], axis=1)
    return data

flanker_df = create_flanker_dataframe()
data_count = flanker_calculate_responce_time(flanker_df)
print(data_count)
data = calculate_flanker_standard_error(flanker_df, 'correct', "response-time")
print(data)


The number of type answer = not-answer was considered as outlier. 

```
result:
participant  type       
orange       control        2.0
             dehydration    1.0
Name: correctness, dtype: float64
```

#### Flanker Test plots

In [ ]:
def show_plot(data, title, max_y=100, x_label="", y_label="", palette=colors, factors=["dehydration", "control"]):
    """
    This function is defined to be used by other plots that need Bokeh library

    Args:
        data (Dataframe): datafarme used to plot
        title (Str): description of plot
        x_label (str, optional): description of x axis
        y_label (str, optional): description of y axis
        palette (Dict): colors used in plot
        factors (list): description of categories, Defaults to ["dehydration", "control"].

    Returns:
        p: plot
    
    Author(s):
        Roya Gharehbeiklou
    """
    index_cmap = factor_cmap('x', palette=palette, factors=factors, start=1, end=2)
    x = list(data.index.values)
    data_map = {
        'x': x,
        'counts': data.tolist()
        }

    source = ColumnDataSource(data=data_map)
    p = figure(x_range=FactorRange(*x), y_range=(0, max_y), width=plot_width, height=plot_height, title=title, x_axis_label=x_label, y_axis_label=y_label)

    p.vbar(x='x', top='counts', source=source, fill_color=index_cmap, line_color=line_color)

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None
    return p

def plot_standard_error(plot, data):
    """
    This function is used to show error plot

    Args:
        plot (plot): plot
        data (dataframe): data which is intended to be drawn

    Returns:
        plot: plot
    
    Author(s):
        Roya Gharehbeiklou
    """
    x = list(data.index.values)
    data_map = {
        'x': x,
        'upper': data["upper"].tolist(),
        'lower': data["lower"].tolist()

        }
    source = ColumnDataSource(data=data_map)

    w = Whisker(source=source, base="x", upper="upper", lower="lower", 
            line_color='black', level="overlay")
    w.upper_head.line_color = 'black'
    w.lower_head.line_color = 'black'
    w.upper_head.size = w.lower_head.size = 20
    plot.add_layout(w)
    return plot


In [ ]:
def flanker_plot_count(answer_type="correct"): 
    """
    This function shows the answer types for each participant per session

    Args:
        answer_type (str, optional): answer types of 'correct' or 'incorrect'. Defaults to "correct".

    Returns:
        p: plot
    Author(s):
        Roya Gharehbeiklou
    """
    flanker_df = create_flanker_dataframe()
    data = flanker_calculate_counts(flanker_df, answer_type)
    return show_plot(data, f"Flanker Test _ Average of {answer_type} answers", 50, "participant/session", "count")

answer_types =['correct','incorrect']
flanker_count_plot = flanker_plot_count()
flanker_counts = pn.interact(flanker_plot_count, answer_type = answer_types)
flanker_counts

In [ ]:
def flanker_plot_percentage(answer_type="correct"): 
    """
    This function shows the answer types(percentagwise) for each participant per session

    Args:
        answer_type (str, optional): answer types of 'correct' or 'incorrect'. Defaults to "correct".

    Returns:
        p: plot
    Author(s):
        Roya Gharehbeiklou
    """
    flanker_df = create_flanker_dataframe()
    data = calculate_percentage(flanker_df, answer_type)
    return show_plot(data, f"Flanker Test _ Percentage of {answer_type} answers", 100, "participant/session", "Percentage")

answer_types =['correct','incorrect']
flanker_percentage = pn.interact(flanker_plot_percentage, answer_type = answer_types)
flanker_percentage

In [ ]:
def flanker_box_plot():
    """
    This function is defined to draw the box plot using hvplot from pandas 
        that inheritances from Bokeh

    Returns:
        flanker_boxplot: hvplot
    Author(s):
        Roya Gharehbeiklou
    """
    flanker_df = create_flanker_dataframe()
    #dfi = flanker_df.interactive(loc='top').to_dataframe()
    flanker_boxplot = flanker_df[flanker_df['correctness'] == 'correct'][
        [
            'response-time', 
            'participant', 
            'type'
        ]].hvplot.box(
            by='type', 
            groupby='participant',
            title='Reaction time for correct responses',
            xlabel='Session Type', 
            ylabel='Resopnse Time (ms)',height=400, width=400)
    return flanker_boxplot
    
        
flanker_boxplot = flanker_box_plot()
flanker_boxplot

In [ ]:
def flanker_plot_error(answer_type="correct", participants=participants): 
    """
    This function is used to plot errors over counts of correctness column

    Args:
        answer_type (str, optional): correct and incorrect. Defaults to "correct".
        Participants (list[str] , optional): per participant

    Returns:
        p: plot
    Author(s):
        Roya Gharehbeiklou
    """
    flanker_df = create_flanker_dataframe()
    flanker_df = flanker_df[flanker_df.participant.isin(participants)]
    data = flanker_calculate_counts(flanker_df, answer_type)
    data_se = calculate_flanker_standard_error(flanker_df, answer_type, "correctness")
    p = show_plot(data, f"Flanker Test _ {answer_type} answers", 70, "participant/session", "Counts" )
    p = plot_standard_error(p, data_se)
    return p


answer_types =['correct','incorrect']
inter_plot = pn.interact(flanker_plot_error, answer_type = ['correct','incorrect'])
inter_plot

In [ ]:
def flanker_plot_error_response_time(participants=participants): 
    """
    This function is used to plot errors over responce time

    Args:
        Participants (list[str] , optional): per participant

    Returns:
        p: plot
    Author(s):
        Roya Gharehbeiklou
    """
    flanker_df = create_flanker_dataframe()
    flanker_df = flanker_df[flanker_df.participant.isin(participants)]
    data = flanker_calculate_responce_time(flanker_df)
    data_se = calculate_flanker_standard_error(flanker_df, answer_type="", column_name="response-time")
    p = show_plot(data, f"Flanker Test - response time", 1000, "participant/session", "time(ms)" )
    p = plot_standard_error(p, data_se)
    return p


# show(flanker_barplot)
inter_plot = flanker_plot_error_response_time()
show(inter_plot)

### Stroop Test  Analysis

In [ ]:
def stroop_test(): # Mahdiye
    '''
    This function creates a tiny dataframe from stroop test data.
    
    Parameter:
    
    Return: Stroop test dataframe
    
    Author: Mahdiye
    '''
    total_dict = create_merged_df(config)
    stroop_df = total_dict['stroop']
    stroop_df.drop(stroop_df.columns[[3,7]], axis=1, inplace=True)
    stroop_df = stroop_df.set_axis(['participant', 'type','repeat','word name','word color',
                                    'name_color match','pressed _key','status','reaction_time'], axis=1)
    return stroop_df

stroop_df = stroop_test()

In [ ]:
def individual_stroop_bar_plot(participant='blue'):
    '''
    This function plots a bar_plot per participant.
    
    Parameter: participant: 'blue','red','orange','green','pink'
    
    Return: bar plot
    
    Author: Mahdiye
    '''
    stroop_df = stroop_test()
    df = stroop_df[stroop_df['participant']==participant]
    
    dff= df.groupby('type').min().reset_index()
    p = figure(x_range=dff['type'], height=350, 
               title=f'Stroop Test {participant}', y_axis_label="Reaction time(milliseconds)")
    p.vbar(x=dff['type'], bottom=0,top=dff['reaction_time'], width=0.5, line_color='black', color=participant)
    return p

#interactive plots
participants_color =['blue','red','orange','green','pink']
inter_plot = pn.interact(individual_stroop_bar_plot, participant = participants_color)

In [ ]:
def individual_stroop_box_plot(participant):
    '''
    This function plots a box_plot per participant.
    
    Parameter: participant: 'blue','red','orange','green','pink'
    
    Return: bar plot
    
    Author: Mahdiye
    '''
    
    stroop_df = stroop_test()
    df = stroop_df[stroop_df['participant']==participant]
    kinds = df['type'].unique()
    
    # compute quantiles
    qs = df.groupby('type').reaction_time.quantile([0.25, 0.5, 0.75])
    qs = qs.unstack().reset_index()
    qs.columns = ['type', "q1", "q2", "q3"]
    df = pd.merge(df, qs, on='type', how="left")

    # compute IQR outlier bounds
    iqr = df.q3 - df.q1
    df["upper"] = df.q3 + 1.5*iqr
    df["lower"] = df.q1 - 1.5*iqr

    source = ColumnDataSource(df)

    p = figure(x_range=kinds,y_range=[-100,stroop_df['reaction_time'].max() * 1.3],
                title="box plot of stroop test "+participant,
               background_fill_color="#eaefef", y_axis_label="Reaction time(milliseconds)")


    # outlier range
    whisker = Whisker(base='type', upper="upper", lower="lower", source=source)
    whisker.upper_head.size = whisker.lower_head.size = 20
    p.add_layout(whisker)

    # quantile boxes
    p.vbar('type', 0.5, "q2", "q3", color = participant,bottom=0, source=source, line_color="black")
    p.vbar('type', 0.5, "q1", "q2", color=participant, bottom=0, source=source, line_color="black")
    
    # outliers
    outliers = df[~df.reaction_time.between(df.lower, df.upper)]
    p.scatter('type', 'reaction_time', source=outliers, size=6, color="black", alpha=0.5)

    p.xgrid.grid_line_color = None
    p.axis.major_label_text_font_size="14px"
    p.axis.axis_label_text_font_size="12px"

    return p
    
#interactive plots
participants_color =['blue','red','orange','green','pink']
stroop_boxplot = pn.interact(individual_stroop_box_plot, participant = participants_color)
# stroop_boxplot

#### Stroop error bar for the number of correct answers:

In [ ]:
def status_bar_stroop(participants):
    '''
    This function plots a bar_plot for all participants based on the number of correct answers.
    
    Parameter:
    
    Return: bar plot
    
    Author: Mahdiye
    '''
    
    types = list(stroop_df['type'].unique())
 #     participants = list(stroop_df['participant'].unique())

    df = stroop_df[stroop_df['status'] ==1]
    df = df[df.participant.isin(participants)]
    dff = df.groupby(["participant", "type", "repeat"])["status"].count().reset_index()
    dfff = dff.groupby(["participant", "type"])['status'].mean().reset_index()
    participants = list(dfff['participant'].unique())
    
    de_list = dfff[dfff['type']=='dehydration'].status.to_list()
    co_list = dfff[dfff['type']=='control'].status.to_list()
    
    data = {'participants' : participants,
            'control'   : co_list,
            'dehydration'   : de_list,
            }
    data = pd.DataFrame(data)

    palette = ["skyblue", "salmon"] #colors
    x = [ (participant, test) for participant in participants for test in types ]
    counts = sum(zip(data['control'], data['dehydration']), ()) # like an hstack

    source = ColumnDataSource(data=dict(x=x, counts=counts))
    # plot
    p = figure(x_range=FactorRange(*x), width=plot_width, height=plot_height, title='The number of correct answers of stroop test',
               y_axis_label= ' correct answers count', x_axis_label="participant, session type")

    p.vbar(x='x', top='counts', width=1, source=source, line_color=line_color,
           fill_color=factor_cmap('x', palette=palette, factors=types, start=1, end=2))

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None
    return p

# show(status_bar_stroop())

In [ ]:
stroop_df = stroop_test()
def calculate_status_standard_error(stroop_df):
    '''
    This function calculates upper and lower of whisker for error limits.
    
    parameter: target: one of personal data like:
    "heartrate", "calories", "temperature", "body weight", "fat%", "muscle%"
    
    return: a dataframe contains mean of target, upper and lower columns
    
    Author: Roya (The main author)
            Mahdiye (I had to change some parts to be run for my codes) 
    '''
   
    df = stroop_df[stroop_df['status'] ==1]
    df = df.groupby(["participant", "type", "repeat"])["status"].count().reset_index()    
    df_mean = df.groupby(by=["participant", "type"]).agg(mean=("status", "mean"))
    df_se = df.groupby(by=["participant", "type"]).agg(se=("status", "sem"))
#     print(df_se)
    upper = df_mean["mean"] + 1.96 * df_se["se"]
    lower = df_mean["mean"] - 1.96 * df_se["se"]
    data = pd.concat([upper.rename("upper"), lower.rename("lower")], axis=1)
    return data

def stroop_status_plot_error(participants):
    '''
    This function plots bar_plots with errors on them.
    
    parameter: target: one of personal data like:
    "heartrate", "calories", "temperature", "body weight", "fat%", "muscle%"
    
    return: the main bar plot with errors, related to the target
    
    Author: Roya (The main author)
            Mahdiye (I had to change some parts to be run for my codes) 
    '''
    stroop_df = stroop_test()
    stroop_df = stroop_df[stroop_df.participant.isin(participants)]  # select participants
    data_se = calculate_status_standard_error(stroop_df)
#     print(data_se)
    p = status_bar_stroop(participants)
    p = plot_standard_error(p, data_se)
    return p

show(stroop_status_plot_error(participants))

#### Stroop error bar for reaction time:

In [ ]:
def total_bar_stroop(participants):
    '''
    This function plots a bar_plot for all participants based on the average of reaction time.
    
    Parameter:
    
    Return: bar plot
    
    Author: Mahdiye
    '''
    
    stroop_df = stroop_test()
    df = stroop_df[stroop_df.participant.isin(participants)]  # select participants

    #create a list of different session types
    types = list(df['type'].unique())
    
    dff = df.groupby(['participant','type']).mean().reset_index()
    
    # create a list of participants
    participants = list(dff['participant'].unique())

    #create two list of reaction time regarding session types
    control_mean = list(dff[dff['type'] =='control'].reaction_time)
    dehydration_mean = list(dff[dff['type'] =='dehydration'].reaction_time)

    #create a dictionary of 3 keys and values and then convert into a dataframe
    data = {'participants' : participants,
            'control'   : control_mean,
            'dehydration'   : dehydration_mean,
            }
    data = pd.DataFrame(data)

    palette =  ["skyblue", "salmon"] #colors

    # create a list like:
    # [ ("blue", "control"), ("Ablue", "dehydration"), ("red", "control"), ("red", "dehydration"), ... ]
    x = [ (participant, test) for participant in participants for test in types ]
    counts = sum(zip(data['control'], data['dehydration']), ()) # like an hstack

    source = ColumnDataSource(data=dict(x=x, counts=counts))
    # plot
    p = figure(x_range=FactorRange(*x), y_range=[0, data['dehydration'].max()+200], width=600, height=400, 
                title='Average of reaction time stroop test',
                y_axis_label="Reaction time(milliseconds)",
                x_axis_label="participant, session type")

    p.vbar(x='x', top='counts', width=1, source=source, line_color="black",
           fill_color=factor_cmap('x', palette=palette, factors=types, start=1, end=2))

    p.y_range.start = 0
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None

    return p

In [ ]:
def calculate_stroop_standard_error(stroop_df, participants):
    '''
    This function calculates upper and lower of whisker for error limits.
    
    parameter: target: one of personal data like:
    "heartrate", "calories", "temperature", "body weight", "fat%", "muscle%"
    
    return: a dataframe contains mean of target, upper and lower columns
    
    Author: Roya (The main author)
            Mahdiye (I had to change some parts to be run for my codes) 
    '''
    stroop_df = stroop_df[stroop_df.participant.isin(participants)]
    df_mean = stroop_df.groupby(by=["participant", "type"]).agg(mean=("reaction_time", "mean"))
    df_se = stroop_df.groupby(by=["participant", "type"]).agg(se=("reaction_time", "sem"))
    upper = df_mean["mean"] + 1.96 * df_se["se"]
    lower = df_mean["mean"] - 1.96 * df_se["se"]
    data = pd.concat([upper.rename("upper"), lower.rename("lower")], axis=1)
    return data

def stroop_plot_error_bar(participants):
    '''
    This function plots bar_plots with errors on them.
    
    parameter: target: one of personal data like:
    "heartrate", "calories", "temperature", "body weight", "fat%", "muscle%"
    
    return: the main bar plot with errors, related to the target
    
    Author: Roya (The main author)
            Mahdiye (I had to change some parts to be run for my codes) 
    '''    
    data_se = calculate_stroop_standard_error(stroop_df, participants)
    p = total_bar_stroop(participants)
    p = plot_standard_error(p, data_se)
    return p

stroop_rt_barplot = stroop_plot_error_bar(participants)
show(stroop_rt_barplot)

### Stop Signal Analysis

In [ ]:

column_meanings = {'Column':[0,1,2,3,4,5,6,7],
                   'Meaning':['trial type (go or nogo)', 
                              'required response (left or right)', 
                              'when the stop signal is shown (or 0 if not)', 
                              'response time 1', 
                              'status 1 (1=correct, 2=wrong, 3=timeout)',
                              'response time 2 (only in no go trials)',
                              'status 2 (only in no go trials; 1=correct, 2=wrong, 3=timeout)',
                              '1=trial is correct ; 0=trial is not correct']} 

column_meanings = pd.DataFrame(column_meanings)
column_meanings.set_index('Column', inplace=True)
column_meanings


In [ ]:
def stop_test(stop_df): # Jacob
    
    # renaming and reordering columns
    stop_df.rename(columns = {0:'trial_type', 1:'correct_resp.', 
                            2:'stop_signal_delay', 3:'response_time',
                            4:'status', 5:'resonse_time_nogo',
                            6:'status_nogo', 7:'correct'}, inplace = True)

    stop_df = stop_df[['participant', 'type', 'repeat', 'trial_type',
                    'correct_resp.', 'correct', 'response_time',
                    'status', 'stop_signal_delay', 'resonse_time_nogo',
                    'status_nogo']]

    # The average resonse time for go trials per trial type
    avg_go_resp_time = stop_df[stop_df['trial_type'] == 'go'].groupby([
        'participant', 'type','status']).mean()['response_time']


    # The average resonse time for no-go trials per correct/incorrect trial
    avg_nogo_resp_time = stop_df[stop_df['trial_type'] == 'nogo'].groupby([
        'participant', 'type','status_nogo']).mean()['response_time']

    # Good to keep in mind that here, status three corresponds with a correct trail
    # Since there was no press in a no-go trial.

    # Number of errors and time-outs in go trials
    errors_timeout_go = stop_df[(stop_df['trial_type'] == 'go') & 
                                (stop_df['status'] != 1.0)].groupby([
                                    'participant', 'type', 'repeat','status']).count()['trial_type']

    # Number of errors and time-outs in no-go trials
    errors_timeout_nogo = stop_df[stop_df['trial_type'] == 'nogo'].groupby([
        'participant', 'type', 'repeat','status_nogo']).count()['trial_type']
    
    stop_signal_boxplot = stop_df[(stop_df['trial_type'] == 'go') & (
                                   stop_df['correct'] == 1)][
                                       ['response_time', 'participant', 'type']
                                       ].hvplot.box(by=['participant', 'type'], 
                                                    #groupby='participant',
                                                    title='Reaction time for correct responses',
                                                    xlabel='Session Type',
                                                    ylabel='Resopnse Time (ms)',
                                                    rot=40)
                                    
    
    return (avg_go_resp_time, avg_nogo_resp_time, 
            errors_timeout_go, errors_timeout_nogo,
            stop_signal_boxplot, stop_df)

# callig the function
(avg_go_resp_time, avg_nogo_resp_time,
 errors_timeout_go, errors_timeout_nogo,
 stop_signal_boxplot, stop_df) = stop_test(data_dict['stop'])

#and the error bart
avg_go_resp_bar = pf.plot_error_bar(stop_df, 'response_time',
                                    participants=participants,
                                    title='Stop Signal Task - Average response time in go trials per participant, per session type', 
                                    xlabel='Participant, Session type',
                                    ylabel='Average response time (ms)')

show(avg_go_resp_bar)


In [ ]:
stop_signal_boxplot

### Verbal Fluency Analysis

In [ ]:
def verbal_test(verbal_df): # Jacob
    """
    This function takes the verbal dataframe from the data_dict, turns it into
    its own df, calculates average words produced per participant and plots this
    
    Parameters
        verbal_df: The data_dict with verbal as key
        
    Returns
        verbal_df: The verbal task DataFrame
        verbal_avg: A dataframe with the average words produced per participant
                    and session type
        verbal_barplot: A barplot displaying the verbal_avg data.
    """
    
    verbal_df = data_dict['verbal'].copy()
    verbal_df = verbal_df[verbal_df[1] != 'word count'] # to remove silly headers
    verbal_df.rename(columns={0:'word_type', 1:'n'}, inplace=True)
    verbal_df['n'] = verbal_df['n'].astype(int)

    verbal_avg = verbal_df.groupby(['participant', 'type']).mean().round(2)
    
    verbal_avg = verbal_df.groupby(['participant', 'type']).mean().round(2)
    
    return verbal_df, verbal_avg

# Calling the function
verbal_df, verbal_avg = verbal_test(data_dict['verbal'])

# Creating a barplot
verbal_barplot = pf.plot_error_bar(verbal_df, 'n',
                                   participants=participants,
                                   title='Verbal Fluency Task - Number of words produced per participant, per session type',
                                   xlabel='Participant, Session type',
                                   ylabel='Average words produced')

# Now showing it.
show(verbal_barplot)

### Digit Span Analysis

In [ ]:
def digit_test(digit_df):
    '''
    Function to analyse the Digit Span preprocessed data. Calculates the mean 
    and the standar errors needed to plot, makes some calculation per columns to
    better understard the results. 
    
    Arguments:
    digit_df: Merged dataframe with the digit span results preprocessed from every
    patient. It is obtained from the dictionary created in the 'create_merged_df' 
    function and the key "digit".   

    Returns:
    A dataframe with the mean and standar error for 3 different analyses 
    (sequence lenght, number of errors made and clicks difference).
    
    Author:
    Karina Diaz
    '''
    # Change data types
    digit_df = digit_df.astype({'participant': 'string',
                                'type': 'string',
                                'repeat': 'int',
                                'seq length':'float',
                                'errors': 'float',
                                'clicks expected': 'float',
                                'clicks observed': 'float'})

    digit_df['clicks difference'] = digit_df['clicks observed'] - digit_df['clicks expected']

    # Make calculations by column taking the groups into account               
    digit_grouped = digit_df.groupby(['participant','type', 'repeat']).agg(
                                                    {'seq length': 'max',
                                                    'errors': 'mean', 
                                                    'clicks difference':'mean'})

    # Deleate -1 to the longest sequence because that does not count for the analysis
    digit_grouped['seq length'] = digit_grouped['seq length'] - 1

    # Obtain mean and estandar error
    digit_mean_sem = digit_grouped.groupby(['participant', 'type']).agg(['mean','sem'])

    return digit_mean_sem

In [ ]:
#Running digit_test function
digit_mean_sem = digit_test(digit_df = data_dict["digit"])

def digit_barplots(analysis, participants):
    '''Function to plot the digit span data with error bars.

    Arguments:
    analysis: Either 'seq length' or 'errors'. Depending on the word is the 
    barplot that is going to be created.

    Returns:
    Amn object 'p' with the information to create a barplot with error bars of 
    the digit span test. 
    
    Author:
    Karina Diaz
    '''


    df = digit_mean_sem[digit_mean_sem.index.get_level_values(0).isin(participants)]
    # Data for the barplots
    participants = df.reset_index().participant.unique().tolist()
    sessions_type = df.reset_index().type.unique().tolist()
    values = df[analysis]['mean'].tolist()

    # Data for the error bars
    upper = df[analysis]['mean'] + 1.96 * df[analysis]['sem']
    lower = df[analysis]['mean'] - 1.96 * df[analysis]['sem']
    data = pd.concat([upper.rename("upper"), lower.rename("lower")], axis=1)

    # Dictionary to change the y labels
    y_label = {'seq length': 'Number of digits',
              'errors':'Number of errors ',
              'clicks difference': 'Diference in errors made'}
    # Dictionary to change the plot titles
    title = {'seq length': 'Digit Span - Longest average sequence remembered',
              'errors':'Digit Span - Average number of errors made',
              'clicks difference': 'Digit Span - Average diference in errors made'}

    x = [(participant, session) for participant in participants for session in sessions_type]
    source = ColumnDataSource(data=dict(x=x, counts=values))
    
    # Create the barplots
    p = figure(x_range=FactorRange(*x), height=400, title=title[analysis])
    # Customise barplots
    p.vbar(x='x', top='counts', source=source, line_color=line_color,
        fill_color=factor_cmap('x', palette=colors, factors=sessions_type, 
        start=1, end=2))

    # Customise x-axis
    p.xaxis.axis_label = "Participant, Sesion type"
    p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None

    # Customise y-axis
    p.yaxis.axis_label = y_label[analysis]
    p.y_range.start = 0
    p.yaxis.major_label_orientation = "vertical"
    p.y_range.range_padding = 1
    
    # Run the function to add the error bars
    p = plot_standard_error(plot=p, data=data)
    return (p)

digit_seq_plot = digit_barplots('seq length', participants)
show(digit_seq_plot )

digit_error_plot = digit_barplots('errors', participants)
show(digit_error_plot)

# # Making interactive the plots 
#analyses = ['seq length', 'errors']
#inter_plot = pn.interact(digit_barplots, analysis=analyses)
#inter_plot

# # Creating a dashboard
# dashboard = pn.template.BootstrapTemplate(title='Title', sidebar_with = 400)
# dashboard.sidebar.append(inter_plot[0])
# dashboard.main.append(inter_plot[1])
# dashboard.show()


## Panel

To visualise all the results a dashboard i made using Panel (holoviz). The idea was to have a different page for each test and a page for the health data. This idea is implemented using a Dashboard class, where the user can add pages with contents. It will automatically create a sidebar navigation when a page is added. The class can also be used in other projects to create other dashboards.

In [ ]:
sample_text = '''
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque augue eros, tristique ut eros et, bibendum mattis tellus. Integer dui sapien, pulvinar nec ante nec, rutrum feugiat massa. Fusce tristique viverra nunc, sed commodo orci rhoncus sed. Aliquam pellentesque dui lectus, vel gravida eros volutpat vitae. Aliquam faucibus nulla id dolor suscipit elementum. Donec sed ante hendrerit, porta ligula faucibus, venenatis mi. Donec id imperdiet neque. Ut vel blandit urna. Fusce convallis, eros at suscipit aliquam, quam tellus pharetra est, ultrices ultrices dolor mi eu enim. Integer sed rutrum tellus.

Etiam non commodo sem. Fusce faucibus tristique mauris, et fermentum quam euismod et. Vestibulum tempor mi neque, et consectetur odio tincidunt ut. Nunc scelerisque sed neque vitae efficitur. Nulla rutrum purus hendrerit, posuere massa ut, pharetra mi. Pellentesque nisi ipsum, pretium ut interdum eget, tempor at dui. Vestibulum a lectus est. Curabitur faucibus id neque ut pharetra. Proin rutrum aliquet scelerisque. Vestibulum id felis at eros accumsan commodo. Vestibulum nec sem felis. Aenean in ullamcorper diam.

In commodo nisl turpis, id laoreet elit suscipit eu. Mauris ut interdum odio. Vivamus ultricies lorem ligula, ut consequat sapien tempor non. Aenean pellentesque nulla sit amet sem fermentum auctor. Nulla facilisi. Sed iaculis vehicula neque, sit amet tempor libero fringilla quis. Phasellus malesuada placerat elit nec vestibulum. Etiam eu odio imperdiet, ornare leo sed, suscipit magna. Proin diam ante, imperdiet eu odio ac, consectetur euismod ipsum. Vivamus non odio aliquet, dapibus elit sit amet, viverra diam. Proin posuere orci eget orci tempus, ut eleifend ipsum mattis. Fusce ultrices est vitae nibh aliquet sollicitudin. Duis vehicula erat turpis, ac efficitur turpis sagittis eget.

Proin eros sapien, vestibulum at congue a, hendrerit sed lacus. Mauris aliquet egestas mauris, sit amet mattis velit faucibus convallis. Phasellus aliquam sapien eros, quis volutpat velit faucibus ut. Vestibulum pulvinar mollis orci vel fringilla. In dapibus, mi iaculis ornare tincidunt, lacus risus sollicitudin tortor, blandit eleifend tellus arcu id tortor. Cras nec fringilla nunc, a fermentum urna. Vivamus urna ligula, tempus nec dolor sed, fermentum faucibus velit. Nulla convallis vitae turpis in tempor.

Integer non faucibus mi, vel gravida felis. Suspendisse vel mi felis. Curabitur dapibus enim ullamcorper consequat vulputate. Suspendisse scelerisque nibh ut luctus iaculis. Sed nunc urna, hendrerit vel sapien nec, imperdiet posuere felis. Cras varius nibh sed tortor congue, et egestas velit lacinia. Mauris purus magna, posuere vel metus non, tempus mattis lacus. Vestibulum turpis justo, posuere nec ante at, facilisis tristique dui. Aenean gravida, eros in luctus lobortis, ipsum lorem ornare felis, vel volutpat ipsum metus vitae erat. Aliquam condimentum aliquam ipsum, at aliquet quam congue quis. Phasellus eu metus velit.
'''

sample_text_small = '''
Lorem ipsum dolor sit amet, consectetur adipiscing elit. Pellentesque augue eros, tristique ut eros et, bibendum mattis tellus. Integer dui sapien, pulvinar nec ante nec, rutrum feugiat massa. Fusce tristique viverra nunc, sed commodo orci rhoncus sed. Aliquam pellentesque dui lectus, vel gravida eros volutpat vitae. Aliquam faucibus nulla id dolor suscipit elementum. Donec sed ante hendrerit, porta ligula faucibus, venenatis mi. Donec id imperdiet neque. Ut vel blandit urna. Fusce convallis, eros at suscipit aliquam, quam tellus pharetra est, ultrices ultrices dolor mi eu enim. Integer sed rutrum tellus.

Etiam non commodo sem. Fusce faucibus tristique mauris, et fermentum quam euismod et. Vestibulum tempor mi neque, et consectetur odio tincidunt ut. Nunc scelerisque sed neque vitae efficitur. Nulla rutrum purus hendrerit, posuere massa ut, pharetra mi. Pellentesque nisi ipsum, pretium ut interdum eget, tempor at dui. Vestibulum a lectus est. Curabitur faucibus id neque ut pharetra. Proin rutrum aliquet scelerisque. Vestibulum id felis at eros accumsan commodo. Vestibulum nec sem felis. Aenean in ullamcorper diam.
'''

In [ ]:
# # create participant options buttons
# red_btn = pn.widgets.Toggle(name='Red', value=True, width=100, css_classes=['red_button'])
# orange_btn = pn.widgets.Toggle(name='Orange', value=True, width=100, css_classes=['orange_button'])
# green_btn = pn.widgets.Toggle(name='Green', value=True, width=100, css_classes=['green_button'])
# blue_btn = pn.widgets.Toggle(name='Blue', value=True, width=100, css_classes=['blue_button'])
# pink_btn = pn.widgets.Toggle(name='Pink', value=True, width=100, css_classes=['pink_button'])

In [ ]:
class Dashboard:
    '''
    This class creates a panel dashboard to which pages can be added
    
    Arguments: 
    title (str): title of the dashboard
    header_color (str): name of a color or hex color code
    css (str): raw css
    
    Returns:    
    dashboard object
    
    Author(s): 
    Job Maathuis
    '''
    
    def __init__(self, title: str, header_color: str, css):
        # initialise dashboard
        self.dashboard = pn.template.BootstrapTemplate(title=title, header_background=header_color, sidebar_width=200)
        self.dashboard.main.extend([pn.pane.Markdown(''), pn.Column(width=1000)]) 
        self.main_page = self.dashboard.main[1]
        pn.extension(raw_css=[css])
        
        # variable to save all the pages
        self.pages = {}
        
    def add_page(self, title: str, show_page: bool, *contents):
        ''' Adds a page to the dashboards and create a sidebar navigation button for it '''
        sidebar_button = pn.widgets.Button(name=title, width=150, css_classes=['sidebar_button'])  # create sidebar button
        self.dashboard.sidebar.append(sidebar_button)  # append button to sidebar
        sidebar_button.on_click(self._update_page)  # callback
        self.pages[title] = [*contents]  # add the contents to the page dictionary
        if show_page:
            self._show_page(title)
    
    def _update_page(self, event):
        ''' private callback method to update the page when a sidebar button is clicked '''
        name = event.obj.name  # extract name from event
        self.main_page.clear()  # clear the main page
        self.main_page.append(pn.pane.Markdown(f'##{name}'))  # create title
        self.main_page.extend([item for item in self.pages[name]])  # add all of the contents to the page
        
    def _show_page(self, title: str):
        ''' private method that show the page of the given page title '''
        self.main_page.clear()
        self.main_page.append(pn.pane.Markdown(f'##{title}'))
        self.main_page.extend([item for item in self.pages[title]])
            
    def show(self):
        ''' shows the dashboard ''' 
        self.dashboard.show()

Now the class is made, it can be called. However, first the CSS need to be made, which defines the markup of the page. And also the interactive plots need to be defined, so that they can be added to pages.

In [ ]:
# CSS styling
css = '''
.bk {
    margin-left: 80px;
}

.sidebar_button .bk-btn-group button {
    border-radius: 6px;
    font-weight: bolder;
}

.option_button .bk-btn-default.bk-active {
    background-color: #00dcff38;
    font-weight: bold;
    border-color: black;
}
'''

In [ ]:
def create_toggle():
    return  pn.widgets.CheckButtonGroup(options=['red', 'orange', 'blue', 'green', 'pink'],
                                        value  =['red', 'orange', 'blue', 'green', 'pink'],
                                        width  =600, css_classes=['option_button'])

# create widgets
toggle_digit_length = create_toggle()
toggle_digit_errors = create_toggle()
toggle_verbal       = create_toggle()
toggle_stop         = create_toggle()
toggle_stroop_corr  = create_toggle()
toggle_stroop_rt    = create_toggle()
toggle_flanker      = create_toggle()
radio_btn_flanker   = pn.widgets.RadioButtonGroup(options=['correct', 'incorrect'], value='correct',
                                                  width=600, css_classes=['option_button'])



# bind widgets to plots
digit_barplot_length   = pn.bind(digit_barplots, analysis='seq length', participants=toggle_digit_length)
digit_barplot_errors   = pn.bind(digit_barplots, analysis='errors', participants=toggle_digit_errors)
verbal_barplot         = pn.bind(pf.plot_error_bar, df=verbal_df, datacol='n', 
                                 title='Verbal Fluency Task - Number of words produced per participant, per session type',
                                 xlabel='Participant, Session type', ylabel='Average words produced',
                                 participants=toggle_verbal,)
stop_barplot           = pn.bind(pf.plot_error_bar, df=stop_df, datacol='response_time', 
                                 title='Stop Signal Task - Average response time in go trials per participant, per session type', 
                                 xlabel='Participant, Session type',ylabel='Average response time (ms)',
                                 participants=toggle_stop)
stroop_correct_barplot = pn.bind(stroop_status_plot_error, participants=toggle_stroop_corr)
stroop_rt_barplot      = pn.bind(stroop_plot_error_bar, participants=toggle_stroop_rt)
flanker_barplot        = pn.bind(flanker_plot_error, answer_type=radio_btn_flanker, participants=toggle_flanker)


In [ ]:
participants = ['red', 'orange', 'green', 'blue', 'pink']
targets = ["heartrate", "calories", "temperature"]

hydrohomies_db = Dashboard('HydroHomies', '#00C9FF', css)
hydrohomies_db.add_page('Home',                True, sample_text)
hydrohomies_db.add_page('Stroop test',         False, toggle_stroop_corr, stroop_correct_barplot, sample_text_small,
                                                      toggle_stroop_rt,   stroop_rt_barplot, sample_text_small)
hydrohomies_db.add_page('Stop signal test',    False, toggle_stop, stop_barplot, sample_text_small)
hydrohomies_db.add_page('Flanker test',        False, radio_btn_flanker, toggle_flanker, flanker_barplot, sample_text_small)
hydrohomies_db.add_page('Digit span test',     False, toggle_digit_length, digit_barplot_length, sample_text_small,
                                                      toggle_digit_errors, digit_barplot_errors, sample_text_small)
hydrohomies_db.add_page('Verbal fluency test', False, toggle_verbal, verbal_barplot, sample_text_small)
hydrohomies_db.add_page('Health data',         False, pn.interact(show_personal_plot, participant=participants, target = targets),  sample_text_small, pn.interact(personal_plot_error_bar, target = targets), sample_text_small)
hydrohomies_db.show()

---

## Statistical Part

In this part we want to approve or reject our hypotheses. To achieve this, first we start to gather all the relavant datasets from different tests. then, try to find their distribution (try to make a normalized data set out of them). Finally, we will use the proper statistical test in order to approve or reject our hypotheses.

### Hypotheses

#### Language
We expect the verbal production in words per second to decrease under the influence of dehydration.

#### Executive function
We expect the verbal inhibition accuracy and words per second to both decrease under the influence of dehydration.

#### Complex attention
We expect the selective attention accuracy to decrease under the influence of dehydration.

#### Perceptual-motor function
We expect the psychomotor reaction time in ms to increase and accuracy to decrease under the influence of dehydration.

#### Learning and memory
We expect the recall accuracy to decrease under the influence of dehydration.

### Making a statistical Dataframe
making a statistic database.

In [ ]:
# import required libraries
import numpy as np
import re
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.special import digamma
from scipy.stats import norm, poisson, binom, gamma
from scipy.stats import iqr
from scipy.stats import ttest_rel, ttest_ind, wilcoxon
from statsmodels.stats.proportion import proportions_ztest

make a digit span dataframe

In [ ]:
digit_df = data_dict["digit"]
digit_df = digit_df.astype({'participant': 'string',
                                'type': 'string',
                                'repeat': 'int',
                                'seq length':'float',
                                'errors': 'float',
                                'clicks expected': 'float',
                                'clicks observed': 'float'})
digit_df

making a statistic database.

In [ ]:
def stat_df_maker(flanker_df, stroop_df, stop_df, verbal_df, digit_df):
    """
    explanation: This function makes a dataframe for statistical analysis 
                 based on previous refined dataframes.
    input: flanker_df, stroop_df, stop_df, verbal_df, digit_df
    output: statistical dataframe (stat_df)
    
    """
    # add response time and error from Flanker test
    stat_df = pd.DataFrame(flanker_df.iloc[:,[0,1,2,5,6]].\
        rename(columns = {"correctness":"correctness","response-time":"response"}).\
        groupby('participant'))

    # add response time and error from Stroop test
    stroop_df_slice = pd.DataFrame(stroop_df.iloc[:,[0,1,2,7,8]].\
        rename(columns = {"status":"correctness","reaction_time":"response"}).\
        groupby('participant'))
    stat_df = stat_df.merge(stroop_df_slice, left_on=0, right_on=0)

    # add response time and error from Stop Signal test
    stop_df_slice = pd.DataFrame(stop_df.iloc[:,[0,1,2,6,7]].\
        rename(columns = {"status":"correctness", "response_time":"response"}).\
        groupby('participant'))
    stat_df = stat_df.merge(stop_df_slice, left_on=0, right_on=0)

    # add number of words per test from Verbal Fluency test
    verbal_df_slice = pd.DataFrame(verbal_df.rename(columns = {"n":"response"}).\
        groupby('participant'))
    stat_df = stat_df.merge(verbal_df_slice, left_on=0, right_on=0)

    #add sequence length from Digit Span test
    digit_df_slice = pd.DataFrame(digit_df.iloc[:,:5].rename(columns = {"seq length":"response"}).\
        groupby('participant'))
    stat_df = stat_df.merge(digit_df_slice, left_on=0, right_on=0)    
                                           
    stat_df.columns = ['colour', 'flanker', 'stroop', 'stop', 'verbal', 'digit']

    return stat_df

stat_df = stat_df_maker(flanker_df, stroop_df, stop_df, verbal_df, digit_df)
stat_df.head()

### Distributions
Pearson Distributions

In [ ]:
def pearson_distribution(test, data_type, counter, number):

    """
    lisence: written by Hooman Bahrdo
    explanation: This function gives a pearson distribution based on our data frames.
    input: 1- test name 
           2- data type can be normalized and raw here
           3- counter: number of participant
           4- number: control(0), degydration(1)
    output: 1- normalized distribution
            2- raw (gamma) distribution 
    """
    # catagorize the data set based on its type(dehydration, control)
    df = stat_df[test][counter]
    df_type = pd.DataFrame(df.groupby('type'))

    # for digit span find the maximum sequence length and extract raw 
    # distribution and normalized one
    if test == 'digit':
        df_control = pd.DataFrame(df_type[1][0].groupby('repeat').max())['response']
        df_dehydration = pd.DataFrame(df_type[1][1].groupby('repeat').max())['response']

        # normalized data
        df_difference = ((df_control.iloc[:int((len(df_control)/2))].reset_index() +\
            df_control.iloc[int((len(df_control)/2)):].reset_index()) / 2. -\
            (df_dehydration.iloc[:int((len(df_dehydration)/2))].reset_index() +\
            df_dehydration.iloc[int((len(df_dehydration)/2)):].reset_index()) / 2.).dropna() 
        
        # return normalized distribution for digit test 
        if re.search('^norm', data_type):
            return df_difference['response']   
        
        # return raw distribution (gamma distribution)for digit test  
        else:
            return pd.DataFrame(df_type[1][number].groupby('repeat').max())['response']
    
    # extract raw distribution and normalized one for other datasets
    else:
        df_control = df_type[1][0]['response'].dropna()
        df_dehydration = df_type[1][1]['response'].dropna()

        # make the normalized distribution
        df_difference = ((df_control.iloc[:int((len(df_control)/2))] +\
            df_control.iloc[int((len(df_control)/2)):]) / 2. -\
            (df_dehydration.iloc[:int((len(df_dehydration)/2))] +\
            df_dehydration.iloc[int((len(df_dehydration)/2)):]) / 2.).dropna()  

        # return normalized distribution 
        if re.search('^norm', data_type):
            return df_difference  
        
        # return raw distribution (gamma distribution)
        else:
            return df_type[1][number]['response'].dropna()

Binomial Distribution

In [ ]:
def binomial_distribution(test, counter, number):

    """
    lisence: written by Hooman Bahrdo
    explanation: This function gives properties of binomial distribution (k, p) 
                 for data frames that has TWO states.
    input: 1- test name 
           3- counter: number of participant
           4- number: control(0), degydration(1)
    output: 1- bionomial distribution properties:k, p
    """

    # choose the proper dataframe based on the test name and number of participants
    bin = []
    df = stat_df[test][counter]
    df_type = pd.DataFrame(df.groupby('type'))

    df = df_type[1][number]

    # exempt Flanker test as it has different two state data(correct, incorrect)
    if test == 'flanker':
        incorrect_number = len(df[df['correctness'] != 'correct'])

    else:
        incorrect_number = len(df[df['correctness'] != 1])

    # we can suppose that the number of tests is the k_value if we consider
    # each person seperately as a population
    k = len(df)
    bin.append(k)
    p = incorrect_number / len(df)
    bin.append(p)

    return bin

Gamma Distribution parameters in different methods

In [ ]:
def gamma_parameters(df):

    """
    license: this function is constructed base on Dr. APOL'S CODES.
             but it is converted to a function for specific purpose by Hooman Bahrdo.
    explanation: this function give the a GAMMA dataframe, and calculate all the relevant
                 parameters and variables.
    input: 1- dataframe
    output: 1- avarage (y_av)
            2- mu parameter (Robust method) (mu_R)
            3- probability distribution function (Maximum Likelihood) (f_ML)
            4- probability distribution function (Method of Moments) (f_MM)
            5- x range (x)
            6- ML name (first_method)
            7- MM name (second_method)
    """

    method_parameters_list = []

    # calculate mean value and varians
    first_method ='ML'
    second_method ='MM' 
    y_av = np.mean(df)
    lny_av = np.mean(np.log(df))
    m_2 = np.mean((df - y_av)**2)
    Delta = np.log(y_av) - lny_av

    # calculate robust method parameters
    mu_R = np.median(df) 
    sigma_R = iqr(df)/1.349

    # Define function(s) to find the root(s) of:
    def ML_a(a, Delta):
        return np.log(a) - digamma(a) - Delta

    # MM estimates:
    # sparse data set in digit span test 
    try:
        a_MM = y_av**2 / m_2
        theta_MM = y_av / m_2
    except ZeroDivisionError:
        a_MM = 0
        theta_MM = 0

    method_parameters_list.append(a_MM)
    method_parameters_list.append(theta_MM)

    # Use a_MM as initial guess for a in fsolve function:
    sol = fsolve(ML_a, a_MM, Delta)
    a_ML = sol[0]

    theta_ML = a_ML / y_av


    method_parameters_list.append(a_ML)
    method_parameters_list.append(theta_ML)

    # Plot ML estimate of Gamma(a, theta) distribution:
    x = np.linspace(np.min(df), np.max(df), 501)
    f_ML = np.array([gamma.pdf(xi, a_ML, loc=0, scale=1/theta_ML) for xi in x])

    # sparse data set in digit span test 
    try:
        f_MM = np.array([gamma.pdf(xi, a_MM, loc=0, scale=1/theta_MM) for xi in x])
    except ZeroDivisionError:
        f_MM =0
        
    return y_av, mu_R, f_ML, f_MM, x, first_method, second_method, method_parameters_list

Normal Distribution parameters in different methods

In [ ]:
def normal_parameters(df):

     """
     license: this function is constructed base on Dr. APOL'S CODES.
             but it is converted to a function for specific purpose by Hooman Bahrdo.
     explanation: this function give the a Normal dataframe, and calculate all the relevant
                 parameters and variables.
     input: 1- dataframe
     output: 1- avarage (mu_ML)
            2- mu parameter (Robust method) (mu_R)
            3- probability distribution function (Maximum Likelihood) (f_N_ML)
            4- probability distribution function (Robust Method) (f_N_R)
            5- x range (x)
            6- ML name (first_method)
            7- MM name (second_method)
     """

    # calculate robust method parameters
     first_method ='ML'
     second_method ='R' 
     mu_R = np.median(df) 
     sigma_R = iqr(df)/1.349

    # calculate Maximum Likelihood parameters
     mu_ML = np.mean(df)
     sigma2_ML = np.mean((df - mu_ML)**2)
     sigma_ML = np.sqrt(sigma2_ML)

     # handle ZeroDivisionError when sigma2_ML=0 (lack of data points)
     try:
          s2 = len(df)/(len(df)-1) * sigma2_ML
          s = np.sqrt(s2)
     except ZeroDivisionError:
          s2 = 0
          s = 0
     
     # make x range based on the data frame
     x = np.linspace(df.min()-(df.max()/10), df.max()+(df.max()/10), 201)

     # ML normal distribution:
     f_N_ML = np.array([norm.pdf(xi, loc = mu_ML, scale = s) for xi in x])
     # robust normal distribution:
     f_N_R = np.array([norm.pdf(xi, loc = mu_R, scale = sigma_R) for xi in x])

     return mu_ML, mu_R, f_N_ML, f_N_R, x, first_method, second_method

### Assistant Functions
this function investigate the normality of a distibution

In [ ]:
def finding_normal(df, upper, lower):
     
    """
    license: written by Hooman Bahrdo.
    explanation: this function give the a Normal dataframe, upper and lower bonf of 
                 its 95 CI, and find what pecentage of a dataframe locate inside this
                 interval.
    input: 1- dataframe
           2- upper limit 
           3- lower limit
    output: 1- normality status
    """
    # check whether each point exists between intervals
    i = 0
    for counter, data in enumerate(df):

        if data >= lower[counter] and data <=upper[counter]:
            i += 1

    normal_finder = i/len(df)

    # chcke which range best describes this distribution
    if normal_finder >= 0.95:
        description = 'Normal > 95%'
    elif normal_finder < 0.95 and normal_finder>=0.9:
        description = 'Normal > 90%'
    elif normal_finder < 0.9 and normal_finder >= 0.8:
        description = 'Normal > 80%'   
    else:
        description = 'not Normal'
        
    return description         

This function add label to plots

In [ ]:
def adding_label(data_type, sub_plot, i, j, plot_type):

    """
    license: Written by Hooman Bahrdo
    explanation: This function add labels to the plot
    input:  1- data_type
            2- sub plot 
            3- i: number of raw
            4- j: number of column
            5- type of plot
    output: some labels to x and y axis
    """
    
    # draw labels for normal and raw data  
    if re.search('^norm|^raw', data_type):
        if j == 0 and re.search('^his', plot_type):
            sub_plot[i,j].set(ylabel='frequency')

        elif j == 0 and re.search('^Q_Q', plot_type):
            sub_plot[i,j].set(ylabel='Sample quantiles')

        if i == 1 and re.search('^his', plot_type):
            sub_plot[i,j].set(xlabel='reponse time difference')
                
        elif i == 1 and re.search('^Q_Q', plot_type):
            sub_plot[i,j].set(xlabel='Theoretical quantiles')

    # draw labels for error data  
    else:
        if j == 0:
            sub_plot[i,j].set(ylabel='Probability')

        if i == 1 :
            sub_plot[i,j].set(xlabel='Random Variable')

### Plots
Binomial plot 

In [ ]:
def binomial_plot(df, sub_plot,counter_1, counter_2, color):

    """
    license: this function is constructed base on one of the Dr. APOL'S FUNCTIONS.
             but most of it is written by Hooman Bahrdo.
    explanation: this function give the properties of a binomial distribution and 
                 skethch its plot.
    input: 1- dataframe
           2- sub plot 
           3- counter1: number of raw
           4- counter2: number of column
           5- color of participant
    output: 1- bionomial distribution (k, p) plot
    """

    # Make binomial distribution:
    k, p = df[0], df[1]

    # Normal approximation:
    mu = k*p
    sigma = np.sqrt(k*p*(1-p))

    # Make a plot:
    i = list(range(k+3))
    f_i = np.array([binom.pmf(xi, k, p) for xi in i])
    x = np.linspace(np.min(i),np.max(i)+1,501)

    # if mu>5 and k-mu>5, then we can consider our distribution as a normal distribution.
    # otherwise, we should consider our distribution as poisson distribution. (Bagui & Mehra 2017)
    if mu>5 and k-mu>5:
        f = np.array([norm.pdf(xi, mu, sigma) for xi in x])

    else:
        # Corresponding parameter of the Pois(lambda) distribution:
        lamb = k*p
        f = np.array([poisson.pmf(xi, lamb) for xi in x])

    # set label for index once
    if counter_1 == 0 and counter_2 == 0:
        sub_plot.vlines(i, 0, f_i, colors='k', linestyles='-', lw=1,label='Simulated Distribution')
        sub_plot.plot(x, f, 'r', label='Approximation Curve')
        sub_plot.set_title(color, color=color, fontsize='xx-large') 
        sub_plot.text(0.8, 0.9, f'k= {round(k,2)}, p= {round(p,2)}', horizontalalignment='center',\
                       verticalalignment='center', transform=sub_plot.transAxes, fontsize='xx-large')
        
        # impose x range        
        for number in range(501):
            if f[number] < 1e-3 and x[number] > i[int(k/5)]:
                sub_plot.set_xlim([x[0],x[number+10]])
                break
        
    else: 
        sub_plot.vlines(i, 0, f_i, colors='k', linestyles='-', lw=1)
        sub_plot.plot(x, f, 'r')
        sub_plot.set_title(color, color=color, fontsize='xx-large') 
        sub_plot.text(0.8, 0.9, f'k= {round(k,2)}, p= {round(p,2)}', horizontalalignment='center',\
                       verticalalignment='center', transform=sub_plot.transAxes, fontsize='xx-large')
        
        # impose x range
        for number in range(501):
            if f[number] < 1e-5 and x[number] > i[int(k/5)]:
                sub_plot.set_xlim([x[0],x[number+10]])
                break

    return sub_plot

Histogram

In [ ]:
def histogram(df, sub_plot, color, i, j, data_type):
    
     """
     license: the function's structure is inspired by one of the Dr. APOL'S FUNCTIONS.
             but is written by Hooman Bahrdo.
     explanation: this function gives the distribution and its type, then
                 skethches its plot.
     input: 1- dataframe
            2- sub plot 
            3- color of participant
            4- i: number of raw
            5- j: number of column
            6- data_type: raw or normalized data 
  
     output: 1- histogram and its probability distribution funcion plot
     """     

     if re.search('^norm', data_type):
          mu_ML, mu_R, f_ML, f_R, x, name_1, name_2= normal_parameters(df)
     else:
          mu_ML, mu_R, f_ML, f_R, x, name_1, name_2 = gamma_parameters(df)
     
     # extract the distribution name
     name = re.split('/s', data_type) 
     distribution_name = name[0]

     #make just one lable for the legend
     if i == 0 and j ==0:
          sub_plot.hist(x=df, density=True, bins='auto', 
               color='darkgrey',alpha=1, rwidth=1, label='experimental data')
          sub_plot.set_title(color, color=color, fontsize='xx-large')

          sub_plot.grid(axis='y', alpha=0.5)
          # sparse data set in digit span test 
          try:
               sub_plot.plot(x, f_ML, 'r', label=f'{distribution_name} Distribution {name_1}')
               sub_plot.plot(x, f_R, 'b', label=f'{distribution_name} Distribution {name_2}')
          except ValueError:
               pass
          
          sub_plot.axvline(x = mu_ML, color='red', linestyle='--', label='dataframe mean value')
          sub_plot.axvline(x = mu_R, color='blue', linestyle='--', label='$\mu$ Robust Theory')

     else:
          sub_plot.hist(x=df, density=True, bins='auto', 
               color='darkgrey',alpha=1, rwidth=1)
          sub_plot.set_title(color, color=color, fontsize='xx-large')  
          sub_plot.grid(axis='y', alpha=0.5)

          # sparse data set in digit span test 
          try:
               sub_plot.plot(x, f_ML, 'r')
               sub_plot.plot(x, f_R, 'b')
          except ValueError:
               pass

          sub_plot.axvline(x = mu_ML, color='red', linestyle='--')
          sub_plot.axvline(x = mu_R, color='blue', linestyle='--')    
     
     return sub_plot

Q_Q Plot

In [ ]:
def DS_Q_Q_Plot(y, sub_plot, color, counter_1, counter_2, est = 'robust'):
    """
    * Minor changes by Hooman Bahrdo.
    *
    Function DS_Q_Q_Plot(y, est = 'robust', **kwargs)
    
       This function makes a normal quantile-quantile plot (Q-Q-plot), also known
       as a probability plot, to visually check whether data follow a normal distribution.
    
    Requires:            - 
     
    Arguments:
      y                  data array
      est                Estimation method for normal parameters mu and sigma:
                         either 'robust' (default), or 'ML' (Maximum Likelihood),
                         or 'preset' (given values)
      N.B. If est='preset' than the *optional* parameters mu, sigma must be provided:
      mu                 preset value of mu
      sigma              preset value of sigma
      
    Returns:
      Estimated mu, sigma, n, and expected number of datapoints outside CI in Q-Q-plot.
      Q-Q-plot
      
    Author:            M.E.F. Apol
    Date:              2020-01-06, revision 2022-08-30
    """

    n = len(y)
    
    # Calculate order statistic:
    y_os = np.sort(y)
  
    # Estimates of mu and sigma:
    # ML estimates:
    mu_ML = np.mean(y)
    sigma2_ML = np.var(y)
    sigma_ML = np.std(y) # biased estimate
    s2 = np.var(y, ddof=1)
    s = np.std(y, ddof=1) # unbiased estimate

    # Robust estimates:
    mu_R = np.median(y)
    sigma_R = iqr(y)/1.349
    
    # Assign values of mu and sigma for z-transform:
    if est == 'ML':
        mu, sigma = mu_ML, s
    elif est == 'robust':
        mu, sigma = mu_R, sigma_R
    else:
        print('Wrong estimation method chosen!')
        return()

    
    # Expected number of deviations (95% confidence level):
    n_dev = np.round(0.05*n)
         
    # Perform z-transform: sample quantiles z.i
    z_i = (y_os - mu)/sigma

    # Calculate cumulative probabilities p.i:
    i = np.array(range(n)) + 1
    p_i = (i - 0.5)/n

    # Calculate theoretical quantiles z.(i):
    from scipy.stats import norm
    z_th = norm.ppf(p_i, 0, 1)

    # Calculate SE or theoretical quantiles:
    SE_z_th = (1/norm.pdf(z_th, 0, 1)) * np.sqrt((p_i * (1 - p_i)) / n)

    # Calculate 95% CI of diagonal line:
    CI_upper = z_th + 1.96 * SE_z_th
    CI_lower = z_th - 1.96 * SE_z_th

    # find whether a distribution is normal
    description = finding_normal(z_i, CI_upper, CI_lower)

    # Make Q-Q plot:
    if counter_1 == 0 and counter_2 ==0:
        sub_plot.plot(z_th, z_i, 'o', color='k', label='experimental data')
        sub_plot.plot(z_th, z_th, '--', color='r', label='normal line')
        sub_plot.plot(z_th, CI_upper, '--', color='b', label='95% CI')
        sub_plot.text(0.8, 0.1, description, horizontalalignment='center',\
                    verticalalignment='center', transform=sub_plot.transAxes,\
                    fontsize='xx-large')
    else:
        sub_plot.plot(z_th, z_i, 'o', color='k')
        sub_plot.plot(z_th, z_th, '--', color='r')
        sub_plot.plot(z_th, CI_upper, '--', color='b')
        sub_plot.text(0.8, 0.1, description, horizontalalignment='center',\
                    verticalalignment='center', transform=sub_plot.transAxes,\
                    fontsize='xx-large')

    sub_plot.plot(z_th, CI_lower, '--', color='b')
    sub_plot.set_title(color, color=color, fontsize='xx-large') 
    return sub_plot

### Plot Making Functions
This function make the requested plot

In [ ]:
def making_plots(df, color, sub_plot, plot_type, i, j, data_type):
    
    """
    license: Written by Hooman Bahrdo
    explanation: This function makes sub plots based on plot type and data type
    input:  1- dataframe
            2- color of participant
            3- sub plot 
            4- type of plot
            5- i: number of raw
            6- j: number of column
            7- data_type: raw or normalized data 
  
    output: a plot based on above information
    """

    if plot_type == 'histogram':
        fig = histogram(df, sub_plot, color, i, j,  data_type)

    elif re.search('^Q_Q plot', plot_type):
        estimate = re.split('\s', plot_type)[-1]
        fig = DS_Q_Q_Plot(df, sub_plot, color,i, j, estimate)
    
    elif plot_type == 'binomial plot':
        fig = binomial_plot(df, sub_plot,i, j, color) 

    return fig

this function is a render for making plot

In [ ]:
def render_plot(test='digit', data_type= 'raw response data', plot_type='histogram'):

    """
    lisence: written by Hooman Bahrdo
    explanation: This function opens all files finds those that have data
                 and erase rest of them. thenadd some property to data and 
                 use the plot function to sketch the data.
    input: type of graph, x_axis, y_axis
    output: a set of graphs
    """

    # plot normal data for all participants
    if data_type =='normalized response data':
        fig, sub_plots = plt.subplots(2,3,figsize=(20, 10))

        #erase empty plot
        fig.delaxes(sub_plots[1,2])
        fig.subplots_adjust(left= 0.05, right=0.9, top=0.95, bottom=0.04)

        # move through sub_plots and sketch them
        participant_num = 0
        for counter in range(len(stat_df)):           
            j = counter

            # move through columns
            for i in range(0,2):

                if participant_num >= 5:
                    break

                #sketch the plot
                making_plots(pearson_distribution(test, data_type, counter, j),\
                                stat_df['colour'][participant_num], sub_plots[i,j], plot_type, i, j, data_type)
                participant_num += 1  
             
                # add lable to side graphs
                adding_label(data_type, sub_plots, i, j, plot_type)

            # for column number more than 2 exit the loop
            if j>=2:
                break
    
    # plot error and raw data
    else:
        fig, sub_plots = plt.subplots(2,5,figsize=(35,15))
        fig.subplots_adjust(left=0.05, right=0.9, top=0.95, bottom=0.04)

        # plot error part
        if data_type == 'error':

            # loop through sub_plots and sketch them
            for counter in range(len(stat_df)):
                j = counter
         
                for i in range(0,2): 
                    making_plots(binomial_distribution(test, counter, i),\
                                    stat_df['colour'][counter], sub_plots[i,j], plot_type, j, i, data_type)  

                    # add lable to side graphs
                    adding_label(data_type, sub_plots, i, j, plot_type)    

        # plot raw data
        else:

            # loop through sub_plots and sketch them
            for counter in range(len(stat_df)):
                j = counter
    
                for i in range(0,2):
                    making_plots(pearson_distribution(test, data_type, counter, i),\
                                    stat_df['colour'][counter], sub_plots[i,j], plot_type, j, i, data_type)  
                    
                    # add lable to side graphs
                    adding_label(data_type, sub_plots, i, j, plot_type)  

    # add some properties to the main plot
    fig.suptitle(f'test= {test}, plot type= {plot_type}, data type= {data_type}', fontsize = 'xx-large')

    if data_type == 'normalized response data':
        fig.legend(loc=4, fontsize = 'xx-large') 
 
    else:  
        fig.legend(loc=7, fontsize = 'xx-large')

    fig.tight_layout()
    fig.subplots_adjust(right=0.85) 

To make these plots interactive you can use these titles

In [ ]:
#making an interactive plot (titles for interactive part)
# you can use these titles if you want to make it interactive
tests = ['flanker', 'stroop', 'stop', 'verbal', 'digit']
plot_name =  ['histogram', 'Q_Q plot ML', 'Q_Q plot robust', 'binomial plot']
data_type = ['normalized response data','raw response data','error']
inter_plot = pn.interact(render_plot, test=tests, data_type=data_type, plot_type=plot_name)

### Statistical Tests
Two-sample T-test

In [ ]:
# Make a custom made 2-sample t-test function:

def two_sample_ttest(y_1, y_2, equal_var = False, alternative = 'two.sided'):

    """ 
    license: Written by dr. Apol and revised by Hooman Bahrdo to fit in this program
    explanation: This function calculate t_sample and p_value of a two sample t-test
    input:  1- control dataframe
            2- dehydration dataframe
            3- variance equality
            4- test type (two.sided, less and greater)
  
    output: t sample and p value
    """

    t_samp, p_value = ttest_ind(y_1, y_2, equal_var = equal_var)
    if alternative == 'two.sided':
        p_real = p_value
    if alternative == 'less':
        if t_samp <= 0:
            p_real = p_value/2
        else:
            p_real = 1 - p_value/2
    if alternative == 'greater':
        if t_samp >= 0:
            p_real = p_value/2
        else:
            p_real = 1 - p_value/2
    return t_samp, p_real

Wilcoxon test

In [ ]:
def wilcoxon_test(y_1, y_2, alternative = 'two.sided'):

    """ 
    license: inspired from dr. Apol's function for t test and written by
             Hooman Bahrdo to fit in this program.
    explanation: This function calculate t_sample and p_value of a Wilcoxon test
    input:  1- control dataframe
            2- dehydration dataframe
            3- variance equality
            4- test type (two.sided, less and greater)
  
    output: t sample and p value for Wilcoxon
    """

    t_samp, p_value = wilcoxon(y_1, y_2)
    if alternative == 'two.sided':
        p_real = p_value
    if alternative == 'less':
        if t_samp <= 0:
            p_real = p_value/2
        else:
            p_real = 1 - p_value/2
    if alternative == 'greater':
        if t_samp >= 0:
            p_real = p_value/2
        else:
            p_real = 1 - p_value/2
    return(t_samp, p_real)

Two-sample z-test for proportion

In [ ]:
def proportion_ztest(y_1, y_2, equal_var = False, alternative = 'two.sided'):

    """ 
    license: inspired from dr. Apol's function for t test and written by
             Hooman Bahrdo to fit in this program.
    explanation: This function calculate stat and p_value of a two sample 
                 z test for proportion.
    input:  1- number of incorrect data points
            2- number of total sample
            3- variance equality
            4- test type (two.sided, less and greater)
  
    output: t sample and p value for Wilcoxon
    """

    t_samp, p_value = proportions_ztest(y_1, y_2)
    if alternative == 'two.sided':
        p_real = p_value
    if alternative == 'less':
        if t_samp <= 0:
            p_real = p_value/2
        else:
            p_real = 1 - p_value/2
    if alternative == 'greater':
        if t_samp >= 0:
            p_real = p_value/2
        else:
            p_real = 1 - p_value/2
    return(t_samp, p_real)

Main statistical test function

In [ ]:
def statistical_test(test, data_type, stat_df):

    """ 
    license: written by Hooman Bahrdo.
    explanation: This function gives the test name and data type and make a list of
    stats and p values of statistical tests for each experimental test.
    input:  1- test name
            2- data type
  
    output: stat list p value list for statistical tests
    """

    t_sample_list = []
    p_real_list = []
    t_sample_w_list = []
    p_real_w_list = []
    stat_list = []
    p_value_z_list = []

    # for raw response time calculate two_sample t_test and Wilcoxon test
    if re.search('^raw', data_type):

        # loop over all participant
        for counter in range(len(stat_df)):
            control_df = pearson_distribution(test, data_type, counter, 0)
            dehydration_df = pearson_distribution(test, data_type, counter, 1)

            t_sample, p_real = two_sample_ttest(control_df, dehydration_df, False, 'greater')
            t_sample_w, p_real_w = wilcoxon_test(control_df, dehydration_df, 'greater')

            # add to their lists
            t_sample_list.append(round(t_sample,4))
            p_real_list.append(round(p_real,4))
            t_sample_w_list.append(round(t_sample_w,4))
            p_real_w_list.append(round(p_real_w,4))

        statistical_test_df = pd.DataFrame(
        {'colour': stat_df['colour'],
        't_value_ttest': t_sample_list,
        'p_value_ttest': p_real_list,
        'stat_wilcoxon_test': t_sample_w_list,
        'p_value_wilcoxon_test': p_real_w_list
        })

        return statistical_test_df

    # for error data calculate two sample z test for proportion
    else:
        # loop over all participant
        for counter in range(len(stat_df)):
            bin_1 = binomial_distribution(test, counter, 0)
            bin_2 = binomial_distribution(test, counter, 1)

            # make count and nobs
            incorrect_1 = int(bin_1[0] * bin_1[1])
            incorrect_2 = int(bin_2[0] * bin_2[1])
            count = np.array([incorrect_1, incorrect_2])
            nobs = np.array([bin_1[0], bin_2[0]])

            stat, p_value_z = proportion_ztest(count, nobs, equal_var = False, alternative = 'greater')            
            stat_list.append(round(stat,4))
            p_value_z_list.append(round(p_value_z,4))
        

        statistical_test_df = pd.DataFrame(
        {'colour': stat_df['colour'],
        'stat_ztest': stat_list,
        'p_value_ztest': p_value_z_list,
       })

        return statistical_test_df

### Making tables
making tabel for Gamma distribution parameters

In [ ]:
def making_binomial_tabel(test, data_type, stat_df):

    # make columns of the tabel
    k_control_list = []
    p_control_list = []
    k_dehydration_list = []
    p_dehydration_list = []    

    # loop through each participant
    for counter in range(len(stat_df)):
        bin_control = binomial_distribution(test, counter, 0)
        bin_dehydration = binomial_distribution(test, counter, 1)

        k_control_list.append(round(bin_control[0],4))
        p_control_list.append(round(bin_control[1],4))
        k_dehydration_list.append(round(bin_dehydration[0],4))
        p_dehydration_list.append(round(bin_dehydration[1],4))

    binomial_parameters_tabel = pd.DataFrame(
        {'colour': stat_df['colour'],
        'k_control': k_control_list,
        'p_control': p_control_list,
        'k_dehydration': k_dehydration_list,
        'p_dehydration': p_dehydration_list,
        })
    return binomial_parameters_tabel

making tabel for Gamma distribution parameters

In [ ]:
def making_gamma_parameters_tabel(test, data_type, stat_df):

    # make columns of the tabel
    a_MM_list_control = []
    theta_MM_list_control = []
    a_ML_list_control = []
    theta_ML_list_control = []
    a_MM_list_dehydration = []
    theta_MM_list_dehydration = []
    a_ML_list_dehydration = []
    theta_ML_list_dehydration = []

    # loop through each participant
    for counter in range(len(stat_df)):
        control_df = pearson_distribution(test, data_type, counter, 0)
        dehydration_df = pearson_distribution(test, data_type, counter, 1)

        mu_ML_1, mu_R_1, f_ML_1, f_R_1, x_1, name1_1, name1_2,\
                                    parameters_list_1 = gamma_parameters(control_df)
        mu_ML_2, mu_R_2, f_ML_2, f_R_2, x_2, name2_1, name2_2,\
                                    parameters_list_2 = gamma_parameters(dehydration_df)

        # append the required data to tabel's columns
        a_MM_list_control.append(round(parameters_list_1[0],4))
        theta_MM_list_control.append(round(parameters_list_1[1],4))
        a_ML_list_control.append(round(parameters_list_1[2],4))
        theta_ML_list_control.append(round(parameters_list_1[3],4))
        a_MM_list_dehydration.append(round(parameters_list_2[0],4))
        theta_MM_list_dehydration.append(round(parameters_list_2[1],4))
        a_ML_list_dehydration.append(round(parameters_list_2[2],4))
        theta_ML_list_dehydration.append(round(parameters_list_2[3],4))

    gamma_parameter_df = pd.DataFrame(
        {'colour': stat_df['colour'],
        'a_MM_control': a_MM_list_control,
        'theta_MM_control': theta_MM_list_control,
        'a_ML_control': a_ML_list_control,
        'theta_ML_control': theta_ML_list_control,
        'a_MM_dehydration': a_MM_list_dehydration,
        'theta_MM_dehydration': theta_MM_list_dehydration,
        'a_ML_dehydration': a_ML_list_dehydration,
        'theta_ML_dehydration': theta_ML_list_dehydration
        })
    
    return gamma_parameter_df

---


## Hypotheses

### Complex attention 
#### Hypothesis
We expect the selective attention accuracy and response time to decrease under the influence of dehydration. 

#### Flanker Test
In this section, first we investigate the distribution type, then find some of its features. Finally, implement statistical test to approve or dis approve its hypothesis.
##### Response Time
response time is supposed to increase under the dehydration condition. To investigate this assumption, first the type of distribution is determined.

In [ ]:
# sketch Flanker histogtam for raw data
render_plot(test='flanker', data_type= 'raw response data', plot_type='histogram')

the histogram of raw Flanker test data for response time shows that approximately all the participant data is distributed with a Gamma distribution pattern. Next, Q_Q plot is utilized to investigate this assumption. The mentioned plot can be sketched based on Maximum Likelihood approach or Robust method.

In [ ]:
# sketch Q_Q plot on the Maximum Likelihood method 
render_plot(test='flanker', data_type= 'raw response data', plot_type='Q_Q plot ML')

In [ ]:
# sketch Q_Q plot on the Robust method 
render_plot(test='flanker', data_type= 'raw response data', plot_type='Q_Q plot robust')

Both Q_Q plots confirm the assumption about the data distribution pattern. Consequently, the mentioned distributions can be evaluated with Method of Moments (MM) and Maximum Likelihood (ML) methods. Below, $a$ and $\theta$ parameters for the mentioned methods are calculated.  

In [ ]:
# make a tabel from a and theta parameters of MM and ML
gamma_parameters_df = making_gamma_parameters_tabel('flanker', 'raw response data', stat_df)
gamma_parameters_df

Finaly, based on the data distribution, two statistical tests (2-sample t-test and Wilcoxon signed-rank test) are implemented to evaluate the hypothesis. First, 2-sample t-test can be implemented on the data sets as t test is the best nominate when there is sparse information about the sample. Second, Wilcoxon signed-rank test can be utilized since the control and dehydration data sets for each person are homogonous in terms of their shape.

In [ ]:
# make a tabel of 2_sample t_test and wilcoxon test
statistical_test_df = statistical_test('flanker', 'raw response data', stat_df)
statistical_test_df

For Blue and Green participants the hypothesis is rejected. Also, even though Wilcoxon p value is less than $\alpha$ = 0.05, the p value of the t test is quite more than the $\alpha$ value. It shows that this two distributions share almost the same shape, but the response time did not increase during the hydration experiment. Consequently, the hypothesis is rejected for the mentioned participants. However, Pink is the only participant who has greater response time after imposing the dehydration condition. Therefore, the hypothesis is approved for the mentioned participant. 

##### Accuracy
Acurracy is supposed to decrease under the dehydration condition. To investigate this assumption, first the type of distribution is determined.
Acurracy data set consist of two values: correct and incorrect. Consequently, we can consider the mentioned data set as a binomial distribution. 

In [ ]:
# sketch Flanker binomial plot for error dat 
render_plot(test='flanker', data_type= 'error', plot_type='binomial plot')

binomial distributions can be estimated by a Normal Distribution if $k$ * $p$ > 0.05 and $k$ * (1-$p$) > 5. Otherwise, it can be estimated by a Poisson Distribution (Bagui & Mehra 2017). Below, $p$ and $k$ parameters for each binomial distribution is calculated. 

In [ ]:
# make a tabel for binomial parameters
binomial_parameters = making_binomial_tabel('flanker', 'error', stat_df)
binomial_parameters

Finaly, based on the data distribution, one statistical test (2-sample z-test for proportion) is implemented to evaluate the hypothesis. the mentioned test can be used for binomial distributions to investigate the hypothesis.

In [ ]:
# make a tabel of two sample z test for proportion
statistical_test_df = statistical_test('flanker', 'error', stat_df)
statistical_test_df

For the majority of the participants (Green, Orange, Pink, and Red) no significant change in the number of errors  is detected, so the hypothesis is rejected for them. However, dehydration affected the accuracy of the participant Blue. Thus, the hypothesis is approved for the mentioned participant.